In [1]:
import pandas as pd
import numpy as np
import json
import pymysql
tag_df = pd.read_excel('tag.xlsx')
df = None
with open("../config.json", mode="r", encoding="utf-8") as f:
    config = json.JSONDecoder().decode(f.read())["mysql"]
    config["cursorclass"] = pymysql.cursors.DictCursor
    conn = pymysql.Connection(**config)
    with conn.cursor() as cursor:
        sql = "Select attraction_id, attraction_name, attraction_desc_path from project.attraction where attraction_desc_path !=''"
        cursor.execute(sql)
        result = cursor.fetchall()
        df=pd.DataFrame(result)

In [20]:
import jieba


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.825 seconds.
Prefix dict has been built successfully.


In [38]:
inner = pd.merge(left=df, right=tag_df, left_on='attraction_name', right_on='景點名稱')
inner

,attraction_id,attraction_name,attraction_desc_path,景點名稱,動物,山林,戶外,室內,飲食,歷史,海,在地體驗（DIY）,攝影,運動(腳踏車、衝浪等),親子
0,191,地耕味-玩味蕃樂園,位於花蓮縣新城鄉，由「地耕味集團」打造的復古風觀光工廠。現場設置伴手禮區，輕鬆擁有在地名產。...,地耕味-玩味蕃樂園,0,0,0,1,1,1,0,1,1,0,1
1,192,太魯閣牌樓,這座帶有中國宮殿風格的牌樓，位於錦文橋南端橋頭與中橫公路交會之路口上，匾題「東西橫貫公路」六...,太魯閣牌樓,0,0,1,0,0,1,0,0,0,0,0
2,193,吉安溪臺灣欒樹,想感受季節變換，不用遠離花蓮市區也能看得到！首推位於吉安鄉的楓林步道，步道平緩好走，全長約2...,吉安溪臺灣欒樹,0,1,1,0,0,0,0,0,1,1,1
3,194,山姆先生咖啡館,山姆先生咖啡館，童話屋般的夢幻造型，被網友封為電影《魔戒》夏爾村哈比人小屋，內裡的田園鄉村風...,山姆先生咖啡館,0,0,1,1,1,0,0,0,1,0,1
4,195,介壽新村,介壽新村牆面有許多故事牆彩繪，包括反共復國、保密防謀等標語，以及想翻牆偷香腸孩子的身影，眷舍...,介壽新村,0,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,362,瑞穗溫泉,瑞穗溫泉的泉質十分特殊罕見，全台灣唯一富含鐵、鋇的碳酸鹽泉，當鐵質接觸空氣氧化後，會在水面形...,瑞穗溫泉,0,0,1,0,0,0,0,0,0,0,0
154,363,羅山自行車道,羅山自行車道位於花蓮最南端的富里，由富里農會出發至羅山瀑布風景區，行程結合了特殊的泥火山地形...,羅山自行車道,0,0,1,0,0,0,0,0,0,1,0
155,364,馬太鞍自行車道,馬太鞍部落是花蓮境內面積最大的內陸濕地，也是阿美族的傳統居住地，原住民部落風貌結合濕地特殊的...,馬太鞍自行車道,0,0,1,0,0,0,0,0,0,1,0
156,365,白鮑溪自行車生態步道 暑假戲水樂園,由鯉魚潭風景區出發，經過池南國家森林遊樂區，來到荖溪白鮑溪一帶，沿著溪水上溯，最後抵達白鮑溪...,白鮑溪自行車生態步道 暑假戲水樂園,0,0,1,0,0,0,0,0,0,1,0


In [43]:
word_split = inner.apply(lambda row: set(jieba.cut(str(row["attraction_desc_path"]))), axis=1)

In [44]:
s=word_split.iloc[0]
for i in range(len(word_split)):
    s|=word_split.iloc[i]
d = pd.DataFrame(sorted(s))
d.to_csv("words.csv")

In [45]:
matrix = inner.apply(lambda row: [1 if i in row.attraction_desc_path else 0 for i in d[0]], axis=1)

In [49]:
X=pd.DataFrame([matrix.iloc[i] for i in range(len(matrix))])
X.shape

(158, 7633)

In [50]:
inner.columns

Index(['attraction_id', 'attraction_name', 'attraction_desc_path', '景點名稱',
       '動物', '山林', '戶外', '室內', '飲食', '歷史', '海', '在地體驗（DIY）', '攝影',
       '運動(腳踏車、衝浪等)', '親子'],
      dtype='object')

In [54]:
tags=['動物', '山林', '戶外', '室內', '飲食', '歷史', '海', '在地體驗（DIY）', '攝影','運動(腳踏車、衝浪等)', '親子']

In [51]:
y=inner[tags]
y.shape

(158, 11)

In [52]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y =  train_test_split(X, y, test_size=0.1, random_state=43)

In [55]:
from sklearn.linear_model import LogisticRegression
# Train LogisticRegression Model
classfiers = []
for i, t in enumerate(tags):
    LGR_Classifier = LogisticRegression(n_jobs=-1, random_state=43)
    LGR_Classifier.fit(train_X, train_y[t])
    classfiers.append(LGR_Classifier)

In [57]:
from sklearn import metrics
for c, t in zip(classfiers, tags):
    accuracy = metrics.accuracy_score(test_y[t], c.predict(test_X))
    print(t, accuracy)

動物 1.0
山林 0.6875
戶外 0.6875
室內 0.875
飲食 0.875
歷史 0.8125
海 0.9375
在地體驗（DIY） 0.9375
攝影 0.75
運動(腳踏車、衝浪等) 0.875
親子 0.75


In [58]:
import pickle
for c, t in zip(classfiers, tags):
    pickle.dump(c, open(t, 'wb'))

In [60]:
matrix2 = df.apply(lambda row: [1 if i in row.attraction_desc_path else 0 for i in d[0]], axis=1)

In [63]:
X2=pd.DataFrame([matrix2.iloc[i] for i in range(len(matrix2))])
X2.shape

(601, 7633)

In [64]:
prd = [[r[1]-r[0] for r in c.predict_proba(X2)] for c in classfiers]

In [70]:
prd_df=pd.DataFrame(prd)
prd_df=prd_df.T
prd_df.columns = pd.Series(tags)
prd_df["景點名稱"] = df.attraction_name
prd_df.shape

(601, 12)

In [71]:
prd_df.to_csv("prd")

In [73]:
prd_df.景點名稱

0      地耕味-玩味蕃樂園
1          太魯閣牌樓
2        吉安溪臺灣欒樹
3        山姆先生咖啡館
4           介壽新村
         ...    
596       頭城文創園區
597          太陽埤
598         東澳國小
599      官老爺休閒農場
600      凡梨花休閒農場
Name: 景點名稱, Length: 601, dtype: object

In [75]:
m = prd_df[prd_df.columns[:-1]].to_numpy()

In [76]:
inner_product = np.dot(m, m.T)
inner_product.shape

(601, 601)

In [81]:
length = [np.dot(i, i.T)**0.5 for i in m]

In [82]:
def sim(i, j):
    return inner_product[i][j]/length[i]/length[j]

In [83]:
pd.DataFrame(inner_product).to_csv("inner.csv")
pd.DataFrame(length).to_csv("length.csv")

In [84]:
similarity = [[sim(i, j) for j, name in enumerate(prd_df["景點名稱"])] for i, name in enumerate(prd_df["景點名稱"])]

In [89]:
sim_df=pd.DataFrame(similarity)
sim_df.shape
sim_df.to_csv("sim.csv",index=False)

In [88]:
sim_df

,0,1,2,3,4,5,6,7,8,9,...,591,592,593,594,595,596,597,598,599,600
0,1.000000,0.787676,0.879373,0.326327,0.661267,0.598552,0.792328,0.097223,0.463767,0.103688,...,0.946846,0.958824,0.923120,0.920590,0.939622,0.986741,0.927003,0.959203,0.950553,0.915075
1,0.787676,1.000000,0.757028,0.133071,0.663310,0.836395,0.839096,0.284758,0.648428,0.280399,...,0.778182,0.797032,0.756119,0.795106,0.634259,0.797822,0.790440,0.793505,0.788846,0.736698
2,0.879373,0.757028,1.000000,0.278651,0.461440,0.592364,0.579145,0.266820,0.448890,0.049810,...,0.942528,0.923164,0.929070,0.986791,0.796021,0.928392,0.990908,0.969590,0.903773,0.965443
3,0.326327,0.133071,0.278651,1.000000,0.134246,-0.043607,-0.027538,0.472556,0.463040,0.468718,...,0.398863,0.384507,0.406343,0.349672,0.284484,0.356956,0.286371,0.275313,0.377452,0.381155
4,0.661267,0.663310,0.461440,0.134246,1.000000,0.485836,0.836658,0.283376,0.649395,0.281428,...,0.660891,0.505382,0.635713,0.486763,0.769851,0.569138,0.494585,0.534592,0.741668,0.615233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.986741,0.797822,0.928392,0.356956,0.569138,0.615470,0.724198,0.116016,0.444606,0.123576,...,0.959208,0.985952,0.938633,0.964708,0.890906,1.000000,0.965439,0.982848,0.944597,0.936576
597,0.927003,0.790440,0.990908,0.286371,0.494585,0.609356,0.638661,0.209419,0.448087,0.057012,...,0.957071,0.957086,0.941992,0.996183,0.830836,0.965439,1.000000,0.990298,0.925046,0.965164
598,0.959203,0.793505,0.969590,0.275313,0.534592,0.604822,0.705788,0.173769,0.410598,0.053523,...,0.950275,0.978104,0.925230,0.983253,0.868673,0.982848,0.990298,1.000000,0.926206,0.950109
599,0.950553,0.788846,0.903773,0.377452,0.741668,0.598131,0.724394,0.233924,0.608131,0.207565,...,0.992267,0.899882,0.984811,0.926736,0.933214,0.944597,0.925046,0.926206,1.000000,0.973741
